In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from helper_functions import *

import xgboost as xgb
from xgboost import XGBRegressor

In [2]:
result_df = pd.read_csv('compare_mape.csv')
train = pd.read_csv('train.csv')
result_df.head()

,Country,MAPE,Density,Active,forecast,category,LR_MAPE,SV_MAPE
0,1001,1.504614,3.334431,1417,[3.3091352 3.31727028 3.32801533 3.33693123 3...,LR,0.992271,1.835232
1,1003,4.143341,7.823300,13401,[7.85833788 7.88518858 7.90744925 7.90828514 7...,tf_dense,4.317589,5.230218
2,1005,1.395184,1.206827,239,[1.16141069 1.16513038 1.17019248 1.18426347 1...,SV,1.833943,1.101416
3,1007,5.228748,1.236650,220,[1.22299695 1.22010887 1.22093809 1.21361172 1...,SV,4.259990,3.574082
4,1009,1.442244,1.777708,789,[1.75237012 1.76360607 1.77065146 1.7765491 1...,LR,1.285277,2.116541


In [3]:
train.head()

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243


In [8]:
%%time
mapes = []
smape = []

IDE = train.cfips.unique()
x_train = np.arange(31).reshape((-1,1))
x_test = np.arange(31,39).reshape((-1,1))
# Iterate all the cfips
for i in range(len(IDE)):
    c = result_df['Country'].iloc[i]
    df = train.loc[train.cfips == c]
    last = df.microbusiness_density.values[-9]
    
    xgbr = XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.075)
    
    #xgbr = XGBRegressor(objective='reg:pseudohubererror', 
     #                  n_estimators=4999, 
      #                  learning_rate=0.0075,
       #                 max_leaves = 17, 
        #                subsample=0.50,
         #               colsample_bytree=0.50, 
          #              max_bin=4096,
           #             n_jobs=2,
            #            eval_metric='mae',
             #          )
    
    xgbr.fit(x_train, df['microbusiness_density'][:31])
    preds = xgbr.predict(x_test)
    #print(np.array(preds).shape)
    
    y_test = np.expand_dims(np.array(df['microbusiness_density'][31:]), axis=0)
    results = evaluate_preds(y_test, preds)
    m = results['mape']
    sm = get_smape(np.squeeze(y_test), np.array(preds))
    smape.append(sm)
    mapes.append(m)
    
    

C:\Users\Masa\anaconda3\envs\gpu_enable\godaddy_microbusiness_density_forecasting\helper_functions.py:71: RuntimeWarning: divide by zero encountered in true_divide
  smap[pos_ind] = num[pos_ind] / dem[pos_ind]


CPU times: total: 5min 43s
Wall time: 56.5 s


In [9]:
result_df['XGBR_MAPE'] = mapes
result_df['XGBR_SMAPE'] = smape
mape_table = result_df[['Country', 'MAPE', 'LR_MAPE', 'SV_MAPE', 'XGBR_MAPE', 'XGBR_SMAPE']]
mape_table = mape_table.rename(columns={'MAPE': "tf_dense_MAPE"})
mape_table

,Country,tf_dense_MAPE,LR_MAPE,SV_MAPE,XGBR_MAPE,XGBR_SMAPE
0,1001,1.504614,0.992271,1.835232,2.405483,2.405482
1,1003,4.143341,4.317589,5.230218,5.460416,5.460416
2,1005,1.395184,1.833943,1.101416,0.903252,0.903253
3,1007,5.228748,4.259990,3.574082,4.030214,4.030214
4,1009,1.442244,1.285277,2.116541,2.388119,2.388120
...,...,...,...,...,...,...
3130,56037,8.937141,3.550418,2.450784,2.432020,2.432019
3131,56039,3.437833,2.390681,1.586990,1.633381,1.633382
3132,56041,4.339747,3.925458,1.381605,1.933568,1.933568
3133,56043,2.373714,2.219479,3.608545,3.617006,3.617006


In [10]:
best_scores = mape_table.min(axis='columns')
cat = []
for i, m in enumerate(best_scores):
    if m == mape_table['tf_dense_MAPE'].iloc[i]:
        category = 'tf_dense'
        cat.append(category)
    elif m == mape_table["LR_MAPE"].iloc[i]:
        category = "LR"
        cat.append(category)
    elif m == mape_table['SV_MAPE'].iloc[i]:
        category = "SV"
        cat.append(category)
    else:
        category = 'XGBR'
        cat.append(category)
mape_table['category'] = cat
mape_table.head()

,Country,tf_dense_MAPE,LR_MAPE,SV_MAPE,XGBR_MAPE,XGBR_SMAPE,category
0,1001,1.504614,0.992271,1.835232,2.405483,2.405482,LR
1,1003,4.143341,4.317589,5.230218,5.460416,5.460416,tf_dense
2,1005,1.395184,1.833943,1.101416,0.903252,0.903253,XGBR
3,1007,5.228748,4.259990,3.574082,4.030214,4.030214,SV
4,1009,1.442244,1.285277,2.116541,2.388119,2.388120,LR


In [11]:
mape_table['category'].value_counts()

SV          1138
LR           831
tf_dense     777
XGBR         389
Name: category, dtype: int64

In [12]:
mape_table.describe()

,Country,tf_dense_MAPE,LR_MAPE,SV_MAPE,XGBR_MAPE,XGBR_SMAPE
count,3135.000000,3135.000000,3.135000e+03,3135.000000,3135.000000,3135.000000
mean,30376.037640,27.996707,8.927551e+03,3.909992,164.115509,inf
std,15145.862593,1269.835687,4.995858e+05,21.621562,8962.489258,NaN
min,1001.000000,0.175896,2.424637e-01,0.000000,0.006342,0.006342
25%,18178.000000,1.800255,1.664201e+00,1.394263,1.517019,1.517019
50%,29173.000000,3.101383,2.748950e+00,2.353172,2.523194,2.523194
75%,45076.000000,5.295601,4.815689e+00,3.993313,4.177346,4.177346
max,56045.000000,71091.580000,2.797235e+07,1183.264800,501812.937500,inf


In [41]:
mape_table.to_csv('add_XGBR_results.csv', index=False)

In [48]:
mape_table.to_csv('add_XGBR_result_2.csv', index=False)

In [49]:
d = pd.read_csv('add_XGBR_results.csv')
d.describe()

,Country,tf_dense_MAPE,LR_MAPE,SV_MAPE,XGBR_MAPE
count,3135.000000,3135.000000,3.135000e+03,3135.000000,3135.000000
mean,30376.037640,27.996707,8.927551e+03,3.909992,116.271379
std,15145.862593,1269.835687,4.995858e+05,21.621562,4497.496999
min,1001.000000,0.175896,2.424637e-01,0.000000,0.020653
25%,18178.000000,1.800255,1.664201e+00,1.394263,2.300055
50%,29173.000000,3.101383,2.748950e+00,2.353172,5.577039
75%,45076.000000,5.295601,4.815689e+00,3.993313,89.062245
max,56045.000000,71091.580000,2.797235e+07,1183.264800,251844.530000
